In [1]:
from PIL import ImageTk, Image  

In [2]:

from tkinter import *
from PIL import Image, ImageTk
import pymongo
from tkinter import messagebox
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_tkagg import ( FigureCanvasTkAgg, NavigationToolbar2Tk) 
from matplotlib.figure import Figure
from wordcloud import WordCloud
import pymongo
from pymongo import MongoClient
from tkinter.ttk import Combobox
from tkinter import Toplevel, Text
import seaborn as sns


# Toxins Data Insertion and Data preparation

In [3]:
mechanism = pd.read_csv('Relations_table1.csv')
target = pd.read_csv('AllToxinTarget.csv' , low_memory=False)
edited = pd.read_csv('Mechanism.csv' ,low_memory=False)
structures = pd.read_csv('ToxinStructures.csv')
data =pd.read_csv('AllToxinData.csv')

In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client['toxins']

In [5]:
all_toxin_target_collection = db['AllToxinTarget']
all_toxin_target=target.to_dict(orient='records')
#all_toxin_target
all_toxin_target_collection.insert_many(all_toxin_target)

In [6]:
edited_json_data=edited.to_dict(orient='records')
structure_json_data=structures.to_dict(orient='records')

In [7]:
all_toxin_data_collection = db['AllToxinData']
all_toxin_data=data.to_dict(orient='records')
#all_toxin_data
all_toxin_data_collection.insert_many(all_toxin_data)

In [8]:
all_toxin_mec_collection = db['Mechanism']
all_toxin_mec=mechanism.to_dict(orient='records')
#all_toxin_data
all_toxin_mec_collection.insert_many(all_toxin_mec)

In [9]:
for entry in all_toxin_data:
    del (entry['mechanism_of_toxicity'])

In [10]:
all_toxin_mec_collection.aggregate( [{"$match":{"mechanismID":{ "$ne": "_" }}},
    { "$group" : { "_id" : "$Toxin T3DB ID", "target-mechanism": { "$push": {"targetID":"$Target UniProt ID","mechanismID":"$mechanismID" } } } },
    { "$out" : "authors" }
] )

all_toxin_mec_collection.aggregate( [{"$match":{"mechanismID":{ "$ne": "_" }}},
    { "$group" : { "_id" : "$Target UniProt ID", "toxin-mechanism": { "$push": {"toxinID":"$Toxin T3DB ID","mechanismID":"$mechanismID" } } } },
    { "$out" : "authors1" }
] )

all_toxin_mec_collection.aggregate( [{"$match":{"mechanismID":{ "$ne": "_" }}},
    { "$group" : { "_id" : "$mechanismID", "toxin-target": { "$push": {"toxinID":"$Toxin T3DB ID","targetID":"$targetID" } } } },
    { "$out" : "authors2" }
] )




In [11]:

toxinCollectionData = all_toxin_data_collection.find()
targetCollectionData = all_toxin_target_collection.find()
# Exception here we clear mechanism to be ready for the relation
all_toxin_mec_collection.drop()

all_toxin_mec_collection.insert_many(edited_json_data)

mechanismCollectionData = all_toxin_mec_collection.find()


# recieving the outputs here


authorsCollection = db['authors']

authorsCollection1 = db['authors1']

authorsCollection2 = db['authors2']


for entry in toxinCollectionData:
    
    x = authorsCollection.find({"_id":entry['_id']}).distinct("target-mechanism")
    
    
    all_toxin_data_collection.update_one({"_id":entry['_id']},{"$set":{"target-mechanism":x}})

    


for entry in targetCollectionData:
    x = authorsCollection1.find({"_id":entry['_id']}).distinct("toxin-mechanism")
    
    all_toxin_target_collection.update_one({"_id":entry['_id']},{"$set":{"toxin-mechanism":x}})
    
#     print(x)

for entry in structure_json_data:
    all_toxin_data_collection.update_one({"_id":entry['DATABASE_ID']},{"$set":{"toxin_structure":entry}})



for entry in mechanismCollectionData:
    x = authorsCollection2.find({"_id":entry['_id']}).distinct("toxin-target")
    
    all_toxin_mec_collection.update_one({"_id":entry['_id']},{"$set":{"toxin-target":x}})


In [12]:

import pandas as pd
from pymongo import MongoClient

# Global variables for entry fields and result text widget
old_synonyms_entry = None
new_synonyms_entry = None
result_text = None

# Function to update synonyms in the toxin target collection
def update_synonyms():
    old_synonyms = old_synonyms_entry.get()
    new_synonyms = new_synonyms_entry.get()

    myquery = {"Synonyms": old_synonyms}
    newvalues = {"$set": {"Synonyms": new_synonyms}}

    all_toxin_target_collection.update_one(myquery, newvalues)

    documents = all_toxin_target_collection.find({"Synonyms": new_synonyms}).limit(3)
    result = list(documents)
    df = pd.DataFrame(result)
    result_text.config(state=NORMAL)
    result_text.delete("1.0", END)
    result_text.insert(END, df.to_string(index=False))
    result_text.config(state=DISABLED)

# Function to open the update screen
def open_update_screen():
    global old_synonyms_entry, new_synonyms_entry, result_text  # Make the variables global

    update_screen = Toplevel(root)
    update_screen.title("Update Synonyms")
    update_screen.configure(bg='light blue')  # Set background color to light blue
    update_screen.geometry("400x200")  # Increase screen size

    # Create labels
    old_synonyms_label = Label(update_screen, text="Old Synonyms:", bg='light blue')
    old_synonyms_label.pack()
    new_synonyms_label = Label(update_screen, text="New Synonyms:", bg='light blue')
    new_synonyms_label.pack()

    # Create entry fields
    old_synonyms_entry = Entry(update_screen)
    old_synonyms_entry.pack()
    new_synonyms_entry = Entry(update_screen)
    new_synonyms_entry.pack()

    # Create update button
    update_button = Button(update_screen, text="Update", command=update_synonyms)
    update_button.pack()

    # Create result text widget
    result_text = tk.Text(update_screen, height=10, width=40, state=tk.DISABLED)
    result_text.pack()

# Connect to MongoDB



# Data Insertion 

In [13]:
def insertion_screen():
    global insertion_database
    insertion_database = Toplevel(root)
    insertion_database.title("Toxins")
    insertion_database.geometry("400x200")
    insertion_database.configure(bg="lavender")

    # Create the main text label
    main_label = Label(insertion_database, text="Data Insertion Window", font=("Arial", 14, "bold"), bg="lavender")
    main_label.pack(pady=20)

    # Create the buttons frame
    buttons_frame = Frame(insertion_database, bg="lavender")
    buttons_frame.pack()

    # Create the buttons
    button_all_data = Button(buttons_frame, text="All Toxins Data", width=20, command=dataentry)
    button_all_data.pack(side=LEFT, padx=10)

    button_all_target = Button(buttons_frame, text="All Toxins Target", width=20, command=insert_toxinTarget)
    button_all_target.pack(side=LEFT, padx=10)




In [14]:
def msgbox(msg,title):
    result=messagebox.askokcancel(title=title,message=msg)
    return result

In [15]:
#functions

def save():
    r=msgbox("Save record?","record")
    if r==True:
        newid=mytable.count_documents({})
        if newid!=0:
            
            newid=mytable.find_one(sort=[("_id",-1)])["id"]
        id=newid+1   
        cid.set(id)
        mydict={"_id":int(id.get()),"Name":Name.get(),"studemail":uniprot_name.get(),"student course":ccourse.get()}
        x=mytable.insert_one(mydict)
#delete function

def delete():
    r=msgbox("delete record?","record")
    if r==True:
        myquery={"_id":int(delId.get())}
        mytable.delete_one(myquery)
    
    


### Insert Many functions

In [16]:
def insert_many(mylist):
    # Connect to the MongoDB server and select the database and collection
    client = MongoClient("mongodb://localhost:27017/")
    db = client["toxins"]
    all_toxin_data_collection = db["AllToxinData"]

    # Insert the list of documents into the collection
    all_toxin_data_collection.insert_many(mylist)

def open_insert_window():
    def insert_documents():
        documents = []
        for entry in entries:
            document = {"_id": entry["_id"].get(), "chemical_formula": entry["chemical_formula"].get()}
            documents.append(document)

        insert_many(documents)
        insert_button.config(state=tk.DISABLED)

    # Create insert window
    insert_window = Toplevel(root)
    insert_window.title("Insert Toxin Data")
    insert_window.configure(bg='light blue')
    insert_window.geometry("400x250")

    entries = []

    # Create entry fields
    for i in range(3):
        entry_frame = Frame(insert_window, bg='light blue')
        entry_frame.pack(pady=10)

        _id_label = Label(entry_frame, text="Enter _id:")
        _id_label.pack(side=tk.LEFT)
        _id_entry = Entry(entry_frame)
        _id_entry.pack(side=tk.LEFT)

        chemical_formula_label = Label(entry_frame, text="Enter chemical formula:")
        chemical_formula_label.pack(side=tk.LEFT)
        chemical_formula_entry = Entry(entry_frame)
        chemical_formula_entry.pack(side=tk.LEFT)

        entry = {"_id": _id_entry, "chemical_formula": chemical_formula_entry}
        entries.append(entry)

    # Create insert button
    insert_button = tk.Button(insert_window, text="Insert", command=insert_documents)
    insert_button.pack(pady=20)



In [17]:
from tkinter import Tk, Label, Entry, Button, Toplevel


############all toxins data collection
def update_common_name():
    updateWindow = Toplevel(root)
    updateWindow.title("Update Common Name")
    updateWindow.geometry("400x200")

    old_common_name_label = Label(updateWindow, text="Old Common Name:", bg="lavender", fg="grey")
    old_common_name_label.pack(pady=10)
    old_common_name_entry = Entry(updateWindow)
    old_common_name_entry.pack(pady=10)

    new_common_name_label = Label(updateWindow, text="New Common Name:", bg="lavender", fg="grey")
    new_common_name_label.pack(pady=10)
    new_common_name_entry = Entry(updateWindow)
    new_common_name_entry.pack(pady=10)

    def update():
        old_common_name = old_common_name_entry.get()
        new_common_name = new_common_name_entry.get()

        query = {"common_name": old_common_name}
        new_values = {"$set": {"common_name": new_common_name}}

        try:
            result = all_toxin_data_collection.update_many(query, new_values)
            if result.modified_count > 0:
                messagebox.showinfo("Update Successful", "Common name updated successfully.")
            else:
                messagebox.showwarning("No Records Updated", "No matching records found for update.")
        except Exception as e:
            messagebox.showerror("Update Error", f"An error occurred during update: {str(e)}")

        updateWindow.destroy()

    update_button = Button(updateWindow, text="Update", command=update)
    update_button.pack(pady=10)



def display_max_weight():
    # Call the get_max_weight function to retrieve the maximum weight document
    max_weight_doc = get_max_weight(all_toxin_data_collection)
    
    if max_weight_doc:
        # Create a formatted message to display the result
        message = f"Max Weight Document:\n{max_weight_doc}"
    else:
        message = "No documents found."
    
    # Open a message box to display the result
    messagebox.showinfo("Maximum Weight", message)

def open_update_window():
    update_window = Toplevel(root)
    update_window.title("Update Window")
    update_window.geometry("600x400")

    # Load the background image
    bg_image = ImageTk.PhotoImage(Image.open("find.PNG"))

    # Create a canvas with the background image
    canvas = Canvas(update_window, width=600, height=400)
    canvas.pack(fill="both", expand=True)
    canvas.create_image(0, 0, image=bg_image, anchor="nw")

    # Create buttons in the center of the window
    button1 = Button(update_window, text="update common_name",width=20,bg="#63a7c1",command=update_common_name)
    button1.place(relx=0.5, rely=0.3, anchor=CENTER)

    button2 = Button(update_window, text="update health effect", bg="#63a7c1",command=open_update_window_health_effect)
    button2.place(relx=0.5, rely=0.4, anchor=CENTER)

    button3 = Button(update_window, text="update date", bg="#63a7c1",command=update_toxin_data_gui)
    button3.place(relx=0.5, rely=0.5, anchor=CENTER)

#     button4 = Button(update_window, text="update target data", bg="#63a7c1",command=open_update_target_data)
#     button4.place(relx=0.5, rely=0.6, anchor=CENTER)

    button5 = Button(update_window, text="close", bg="#63a7c1",command=update_window.destroy)
    button5.place(relx=0.5, rely=0.7, anchor=CENTER)




def dataentry():
    dataInsertionwindow = Toplevel(root, bg="lavender")
    dataInsertionwindow.title("Data Insertion Window")
    dataInsertionwindow.geometry("800x700")

    label = Label(dataInsertionwindow, bg="lavender", text="Data Insertion Form", width=30, height=3, anchor="center")
    label.grid(row=0, column=0, columnspan=2, pady=10)

    # Create labels and entry fields
    id_label = Label(dataInsertionwindow, text="ID:", bg="lavender", fg="grey")
    id_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")
    id_entry = Entry(dataInsertionwindow)
    id_entry.grid(row=1, column=1, padx=10, pady=10)

    common_name_label = Label(dataInsertionwindow, text="Common Name:", bg="lavender", fg="grey")
    common_name_label.grid(row=2, column=0, padx=10, pady=10, sticky="e")
    common_name_entry = Entry(dataInsertionwindow)
    common_name_entry.grid(row=2, column=1, padx=10, pady=10)

    description_new_label = Label(dataInsertionwindow, text="Description:", bg="lavender", fg="grey")
    description_new_label.grid(row=3, column=0, padx=10, pady=10, sticky="e")
    description_Entry = Entry(dataInsertionwindow)
    description_Entry.grid(row=3, column=1, padx=10, pady=10)

    chemical_formula_new_label = Label(dataInsertionwindow, text="Chemical Formula:", bg="lavender", fg="grey")
    chemical_formula_new_label.grid(row=4, column=0, padx=10, pady=10, sticky="e")
    chemical_formula_Entry = Entry(dataInsertionwindow)
    chemical_formula_Entry.grid(row=4, column=1, padx=10, pady=10)

    delete_id_label = Label(dataInsertionwindow, text="Delete ID:", bg="lavender", fg="grey")
    delete_id_label.grid(row=5, column=0, padx=10, pady=10, sticky="e")
    delete_id_entry = Entry(dataInsertionwindow)
    delete_id_entry.grid(row=5, column=1, padx=10, pady=10)

    weight_label = Label(dataInsertionwindow, text="Weight:", bg="lavender", fg="grey")
    weight_label.grid(row=6, column=0, padx=10, pady=10, sticky="e")
    weight_entry = Entry(dataInsertionwindow)
    weight_entry.grid(row=6, column=1, padx=10, pady=10)

    def clear_entries():
        id_entry.delete(0, 'end')
        common_name_entry.delete(0, 'end')
        description_Entry.delete(0, 'end')
        chemical_formula_Entry.delete(0, 'end')
        delete_id_entry.delete(0, 'end')
        weight_entry.delete(0, 'end')

    def delete():
        r = messagebox.askyesno("Delete Record", "Are you sure you want to delete the record?")
        if r:
            myquery = {"_id": delete_id_entry.get()}
            all_toxin_data_collection.delete_one(myquery)
            clear_entries()

    def insert_data():
        document = {
            "_id": id_entry.get(),
            "common_name": common_name_entry.get(),
            "description": description_Entry.get(),
            "chemical_formula": chemical_formula_Entry.get(),
            "weight": weight_entry.get()
        }
        all_toxin_data_collection.insert_one(document)
        clear_entries()


    insert_button = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="Save", command=insert_data)
    insert_button.grid(row=7, column=1, padx=10, pady=10, sticky="e")

    delete_button = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="Delete", command=delete)
    delete_button.grid(row=8, column=1, padx=10, pady=10, sticky="e")
    


    #find_btn = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="Find Queries", command=open_find_queries_window)
    #find_btn.grid(row=12, column=1, padx=10, pady=10, sticky="e")

    #update_button = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="Update", command=open_update_window)
    #update_button.grid(row=14, column=1, padx=10, pady=10, sticky="e")
    insert_many_button = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="InsertMany ", command=open_insert_window)
    insert_many_button.grid(row=16, column=1, padx=10, pady=10, sticky="e")
    close_button = Button(dataInsertionwindow, width=25, height=2, bg="grey", fg="white", text="Close", command=dataInsertionwindow.destroy)
    close_button.grid(row=18, column=1, padx=10, pady=10, sticky="e")
    #max_btn = Button(dataInsertionwindow, width=15, height=2, bg="grey", fg="white", text="Find Max Weight", command=display_max_weight)
    #max_btn.grid(row=11, column=1, padx=10, pady=10, sticky="e")


##### find query based on indexing

In [18]:
def indexing_find():
    # Create the indexing_find window
    indexing_window = Toplevel(root)
    indexing_window.title("Indexing Find")
    indexing_window.geometry("800x600")
    indexing_window.configure(bg="lavender")

    # Create labels and radio buttons for sorting order
    sort_label = Label(indexing_window, text="Sort Order:", font=("Arial", 12), bg="lavender")
    sort_label.pack()

    sort_var = StringVar()
    sort_var.set("Ascending")

    ascending_radio = Radiobutton(indexing_window, text="Ascending", variable=sort_var, value="Ascending", bg="lavender")
    ascending_radio.pack()

    descending_radio = Radiobutton(indexing_window, text="Descending", variable=sort_var, value="Descending", bg="lavender")
    descending_radio.pack()

    def show_result():
        sort_order = 1 if sort_var.get() == "Ascending" else -1

        # Create index on the "Name" field
        all_toxin_target_collection.create_index([("Name", sort_order)])

        # Perform the query with the hint for index usage and limit the results to 10
        result = all_toxin_target_collection.find().hint("Name_1" if sort_order == 1 else "Name_-1").limit(10)
        result = list(result)
        df = pd.DataFrame(result)

        # Display the DataFrame result
        result_text.delete("1.0", END)  # Clear previous result
        result_text.insert(END, df)

    # Create a button to show the result
    show_button = Button(indexing_window, text="Show Result", command=show_result, font=("Arial", 12))
    show_button.pack()

    # Create a text widget to display the result
    result_text = Text(indexing_window, height=20, width=100)
    result_text.pack()

#### find query based in toxin state
    

In [19]:
def show_dataframe():
    # Get the selected value from the dropdown
    query = dropdown.get()

    pipeline = [
        {"$match": {"state": query}},
        {"$project": {"_id": 0, "cas": 1, "common_name": 1}},
        {"$limit": 20}
    ]
    result = all_toxin_data_collection.aggregate(pipeline)
    result = list(result)

    df = pd.DataFrame(result)
    df_str = df.to_string()

    # Clear the text widget
    text_widget.config(state=NORMAL)
    text_widget.delete("1.0", END)

    # Insert the DataFrame string into the text widget
    text_widget.insert(END, df_str)

    # Make the text widget read-only
    text_widget.config(state=DISABLED)

def find_State():
    global find_window,dropdown,text_widget
    find_window = Toplevel(root)
    find_window.title("Find Window")
    find_window.geometry("700x500")
    
    find_window.configure(background="lavender")
    
    state = ['Solid', 'Gas', 'Liquid']
    dropdown = Combobox(find_window, values=state)
    dropdown.pack()

    # Set the default selected value
    dropdown.set(state[0])

    show_button = Button(find_window, text="Show",width=20,height=2,bg="plum4" ,command=show_dataframe)
    show_button.pack()

    text_widget = Text(find_window)
    text_widget.pack()

    


##### find query based on molecular weight

In [20]:
def search_database():
    v = float(entry.get())

    col = db["AllToxinTarget"]
    documents = col.find({"Molecular Weight": {"$gt": v}}, {"_id": 0, "NCBI Taxonomy Id": 1, "UniProt Name": 1})
    result = list(documents)
    df = pd.DataFrame(result)

    # Clear the text widget
    text_widget.config(state=NORMAL)
    text_widget.delete("1.0", END)

    # Insert the DataFrame string into the text widget
    text_widget.insert(END, df.to_string())

    # Make the text widget read-only
    text_widget.config(state=DISABLED)

def find_molecule():
    global molecule_window, entry, text_widget

    molecule_window = Toplevel(root)
    molecule_window.title("Find Window")
    molecule_window.geometry("700x500")
    molecule_window.configure(background="lavender")

    entry_label = Label(molecule_window, text="Enter Molecular Weight:")
    entry_label.pack()

    entry = Entry(molecule_window)
    entry.pack()

    show_button = Button(molecule_window, text="Show", width=20, height=2, bg="plum4", command=search_database)
    show_button.pack()

    text_widget = Text(molecule_window)
    text_widget.pack()








##### find query based on common name

In [21]:
def search_database_commonName():
    v = entry.get()

    col = db['AllToxinData']
    documents = col.find({"common_name":v},{"_id":0,'description':1})
    result = list(documents)
    df = pd.DataFrame(result)

    # Clear the text widget
    text_widget.config(state=NORMAL)
    text_widget.delete("1.0", END)

    # Insert the DataFrame string into the text widget
    text_widget.insert(END, df.to_string())

    # Make the text widget read-only
    text_widget.config(state=DISABLED)

def find_commonName():
    global common_name_Window, entry, text_widget

    common_name_Window = Toplevel(root)
    common_name_Window.title("Find bassed on common Name")
    common_name_Window.geometry("700x500")
    common_name_Window.configure(background="lavender")

    entry_label = Label(common_name_Window, text="Enter Molecular Weight:")
    entry_label.pack()

    entry = Entry(common_name_Window)
    entry.pack()

    show_button = Button(common_name_Window, text="Show", width=20, height=2, bg="plum4", command=search_database_commonName)
    show_button.pack()

    text_widget = Text(common_name_Window)
    text_widget.pack()





# Data Query

##### query based on gene bank id

In [22]:
def open_target_viewer():
    # Create toxin target sequence viewer window
    viewer_window = Toplevel(root)
    viewer_window.title("Toxin Target Sequence Viewer")
    viewer_window.geometry("800x700")  # Increase the window size
    viewer_window.configure(bg="#C88EA7")

    # Define the function to execute the query and display the result in a text box
    def execute_query():
        gene_id = input_text.get()
        documents = all_toxin_target_collection.find({"Genbank Gene ID": gene_id}, {"_id": 0, 'Gene Sequence': 1, 'Amino Acid Sequence': 1})
        result = list(documents)
        df = pd.DataFrame(result)
        result_text.configure(state='normal')
        result_text.delete('1.0', tk.END)
        result_text.insert(END, df.to_string(index=False))
        result_text.configure(state='disabled')

    # Create input label and text box
    input_label = Label(viewer_window, text="Enter Genbank Gene ID:", font=("Arial", 12), bg="#C88EA7")
    input_label.pack(pady=10)
    input_text = Entry(viewer_window, font=("Arial", 12), width=40)
    input_text.pack(pady=10)

    # Create button to execute query
    execute_button = Button(viewer_window, text="Execute Query", command=execute_query)
    execute_button.pack(pady=10)

    # Create text box to display result
    result_text = Text(viewer_window, font=("Arial", 12), bg="white", width=100, height=25, wrap=tk.WORD)  # Increase the height
    result_text.pack(pady=10)

    # Create close button
    close_button = Button(viewer_window, text="Close",width=25,height=3 ,command=viewer_window.destroy)
    close_button.pack(pady=10)


In [23]:
import tkinter as tk

def open_data_query_window():

    # Create the main window
    find = Toplevel(root)
    find.title("Data Querying")
    find.configure(bg='steelblue4')

    def open_data_query_window():
        # Function to open the data query window
        window_width = 300
        window_height = 200
        center_window(find, window_width, window_height)

        button_frame = tk.Frame(find, bg='steelblue4')
        button_frame.pack(pady=50)

        find_toxin_data_button = tk.Button(button_frame, text="Find All Toxin Data",
                                          width=25, height=2, command=open_find_queries_window)
        find_toxin_data_button.pack(pady=10)

        find_target_data_button = tk.Button(button_frame, text="Find All Target Data",
                                           width=25, height=2, command=open_find_queries_window)
        find_target_data_button.pack(pady=10)




In [24]:
def open_data_find_window():
    # Create the Toplevel window
    top_window = tk.Toplevel(root)
    top_window.title("Toplevel Window")
    top_window.configure(bg='steelblue4')
    
    window_width = 400
    window_height = 300
    top_window.geometry(f"{window_width}x{window_height}")

    
    # Create Button 1
    button1 = tk.Button(top_window, text="Find All Toxin Data", width=25, height=2, command=open_find_queries_window)
    button1.pack(pady=10)

    # Create Button 2
    button2 = tk.Button(top_window, text="Find All Target Data", width=25, height=2, command=open_find_queries_window_target)
    button2.pack(pady=10)



##### query base on common name indexing

In [25]:
def data_insertion_function():
    # Create the Toplevel window
    top_window = tk.Toplevel(root)
    top_window.title("Data Insertion")
    top_window.configure(bg='steelblue4')
    
    # Set the width and height of the window
    window_width = 400
    window_height = 300
    top_window.geometry(f"{window_width}x{window_height}")

    # Create Button 1
    button1 = tk.Button(top_window, text="toxins data", width=25, height=2, command=dataentry)
    button1.pack(pady=10)

    # Create Button 2
    button2 = tk.Button(top_window, text="target data", width=25, height=2, command=insert_toxinTarget)
    button2.pack(pady=10)
    
    # Create Button 3
    button3 = tk.Button(top_window, text="close", width=25, height=2, command=top_window.destroy)
    button3.pack(pady=10)


###### delete query

In [26]:
import tkinter as tk
from tkinter import messagebox
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["toxins"]
all_toxin_target_collection = db["AllToxinTarget"]

def delete_record_by_molecular_weight():
    # Create a new window for the GUI
    window = tk.Toplevel(root)
    window.title("Delete Record by ID")
    window.geometry("300x150")
    window.configure(bg='white')

    # Function to handle the deletion
    def delete_record():
        molecular_weight = entry.get()

        # Construct the query
        query = {"Molecular Weight": {"$lt": float(molecular_weight)}}

        # Perform the deletion
        result = all_toxin_target_collection.delete_many(query)

        # Display the result in a message box
        if result.deleted_count > 0:
            messagebox.showinfo("Deletion Result", f"{result.deleted_count} record(s) deleted successfully!")
        else:
            messagebox.showinfo("Deletion Result", "No matching record found.")

    # Create the GUI elements
    label = tk.Label(window, text="Enter the Molecular Weight:")
    label.pack(pady=10)

    entry = tk.Entry(window)
    entry.pack(pady=5)

    delete_button = tk.Button(window, text="Delete", command=delete_record)
    delete_button.pack(pady=10)

def delete_one_by_molecular_weight():
    # Create a new window for the GUI
    window = tk.Toplevel(root)
    window.title("Delete Record by Molecular Weight")
    window.geometry("300x150")
    window.configure(bg='white')

    # Function to handle the deletion
    def delete_record():
        molecular_weight = entry.get()

        # Construct the query
        query = {"_id": molecular_weight}

        # Perform the deletion
        result = all_toxin_data_collection.delete_one(query)

        # Display the result in a message box
        if result.deleted_count > 0:
            messagebox.showinfo("Deletion Result", f"{result.deleted_count} record(s) deleted successfully!")
        else:
            messagebox.showinfo("Deletion Result", "No matching record found.")

    # Create the GUI elements
    label = tk.Label(window, text="Enter the Molecular Weight:")
    label.pack(pady=10)

    entry = tk.Entry(window)
    entry.pack(pady=5)

    delete_button = tk.Button(window, text="Delete", command=delete_record)
    delete_button.pack(pady=10) 
import tkinter as tk
from tkinter import messagebox
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["toxins"]
all_toxin_target_collection = db["AllToxinTarget"]

def delete_one_record():
    # Function to handle deleting one record
    # Add your code here for deleting one record
    pass

def delete_many_records():
    # Function to handle deleting many records
    # Add your code here for deleting many records
    pass

def delete_window():
    # Create the delete window
    window = tk.Toplevel()
    window.title("Delete Records")
    window.geometry("300x150")
    window.configure(bg='steelblue4')

    # Create the buttons
    delete_one_button = tk.Button(window, text="Delete One", width=15, command=delete_one_by_molecular_weight)
    delete_one_button.pack(pady=10)

    delete_many_button = tk.Button(window, text="Delete Many", width=15, command=delete_record_by_molecular_weight)
    delete_many_button.pack(pady=10)

    close_button = tk.Button(window, text="Close", width=15, command=window.destroy)
    close_button.pack(pady=10)

    window.mainloop()

def open_delete_window():
    # Function to open the delete window
    delete_window()


In [27]:
import tkinter as tk

def query_window():
    
    # Create the main window
    open_window =Toplevel(root)
    open_window.title("data_quering")
    open_window.geometry("400x300")
    open_window.configure(bg='steelblue4')

    # Create a frame to hold the buttons
    button_frame = tk.Frame(open_window, bg='steelblue4')
    button_frame.pack(pady=50)

    # Create the buttons
    insert_button = tk.Button(button_frame, text="Insert", width=10, bg='grey', command=data_insertion_function)
    insert_button.pack(side=tk.LEFT, padx=10)

    update_button = tk.Button(button_frame, text="Update", width=10, bg='grey', command=open_update_window_entry)
    update_button.pack(side=tk.LEFT, padx=10)

    delete_button = tk.Button(button_frame, text="Delete", width=10, bg='grey', command=open_delete_window)
    delete_button.pack(side=tk.LEFT, padx=10)

    find_button = tk.Button(button_frame, text="Find", width=10, bg='grey', command=open_data_find_window)
    find_button.pack(side=tk.LEFT, padx=10)

    close_button = tk.Button(open_window, text="Close", width=10, bg='grey', command=open_window.destroy)
    close_button.pack(pady=10)




################################################################3
#################################################################3
3#################################################################3
###############################################################3


3

In [28]:
def open_common_name_indexing():
    # Create a new window for the Toxin Data Viewer
    viewer_window = tk.Toplevel(root)
    viewer_window.title("Toxin Data Viewer")
    viewer_window.configure(bg="#C88EA7")
    
    # Define function to execute the query and display the result in a data frame
    def execute_query():
        sorting_order = 1 if sorting_radio.get() == 1 else -1
        result = all_toxin_data_collection.find({}, {"chemical_formula": 1, "state": 1}).sort("common_name", sorting_order).limit(10)
        df = pd.DataFrame(list(result))

        # Display the result in a data frame
        result_text.configure(state='normal')
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, df.to_string(index=False))
        result_text.configure(state='disabled')
    
    # Create sorting order radio buttons
    sorting_radio = tk.IntVar()
    ascending_radio = tk.Radiobutton(viewer_window, text="Ascending", variable=sorting_radio, value=1)
    ascending_radio.pack()
    descending_radio = tk.Radiobutton(viewer_window, text="Descending", variable=sorting_radio, value=-1)
    descending_radio.pack()

    # Create button to execute query
    execute_button = tk.Button(viewer_window, text="Execute Query", command=execute_query)
    execute_button.pack(pady=10)

    # Create text box to display result
    result_text = tk.Text(viewer_window, font=("Arial", 12), bg="white", width=60, height=10, wrap=tk.WORD)
    result_text.pack(pady=10)


In [29]:
def open_find_queries_window_target():
    find_queries_window_target = Toplevel(root)
    find_queries_window_target.title("Find Queries - target data")
    find_queries_window_target.geometry("600x400")

    # Set background image
    background_image = PhotoImage(file="find.PNG")
    background_label = Label(find_queries_window_target, image=background_image)
    background_label.place(x=0, y=0, relwidth=1, relheight=1)

    # Create buttons
    find_mol = Button(find_queries_window_target, text="Find Molecule", width=20, bg="#9c8d91", fg="white", command=find_molecule)
    find_based_on_index = Button(find_queries_window_target, text="Find Based on Index", width=15, height=2, bg="#9c8d91", fg="white", command=indexing_find)
    button3 = Button(find_queries_window_target, text="find commonName Data", width=20, bg="#9c8d91", fg="white", command=open_toxin_viewer)
    find_gene_id = Button(find_queries_window_target, text="genebankID", width=20, bg="#9c8d91", fg="white", command=open_target_viewer)
    button6 = Button(find_queries_window_target, text="common_name indexing", width=20, bg="#9c8d91", fg="white", command=open_common_name_indexing)
    button5 = Button(find_queries_window_target, text="close", width=20, bg="#9c8d91", fg="white", command=find_queries_window_target.destroy)

    # Grid placement of buttons
    find_mol.grid(row=0, column=0, padx=10, pady=10)
    find_based_on_index.grid(row=1, column=0, padx=10, pady=10)
    button3.grid(row=2, column=0, padx=10, pady=10)
    find_gene_id.grid(row=3, column=0, padx=10, pady=10)
    button6.grid(row=4, column=0, padx=10, pady=10)
    button5.grid(row=5, column=0, padx=10, pady=10)




##### find common name data

In [30]:
def open_toxin_viewer():
    # Create toxin information viewer window
    viewer_window = tk.Toplevel(root)
    viewer_window.title("Toxin Information Viewer")
    viewer_window.geometry("800x500")  # Increase the window size
    viewer_window.configure(bg="#C88EA7")

    # Define the function to execute the query and display the result in a text box
    def execute_query():
        common_name = input_text.get()
        documents = all_toxin_data_collection.find({"common_name": common_name}, {"_id": 0, "toxicity": 1, "carcinogenicity": 1, "treatment": 1})
        result = list(documents)
        df = pd.DataFrame(result)
        result_text.configure(state='normal')
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, df.to_string(index=False))
        result_text.configure(state='disabled')

    # Create input label and text box
    input_label = tk.Label(viewer_window, text="Enter Common Name:", font=("Arial", 12), bg="#C88EA7")
    input_label.pack(pady=10)
    input_text = tk.Entry(viewer_window, font=("Arial", 12), width=40)
    input_text.pack(pady=10)

    # Create button to execute query
    execute_button = tk.Button(viewer_window, text="Execute Query", command=execute_query)
    execute_button.pack(pady=10)

    # Create text box to display result
    result_text = tk.Text(viewer_window, font=("Arial", 12), bg="white", width=100, height=20, wrap=tk.WORD)  # Increase the width
    result_text.pack(pady=10)


##### search based on toxin id

In [31]:
def open_toxin_searcher():
    # Create toxin information searcher window
    searcher_window = tk.Toplevel(root)
    searcher_window.title("Toxin Information Searcher")
    searcher_window.configure(bg="#C88EA7")

    # Define the function to search for data in the database
    def search_data():
        search_id = id_entry.get()
        documents = all_toxin_data_collection.find({'_id': search_id}, {'_id': 0, 'metabolism': 1, "cellular_locations": 1,"Locus":1})
        result = list(documents)
        df = pd.DataFrame(result)
        result_text.configure(state='normal')
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, df.to_string(index=False))
        result_text.configure(state='disabled')

    # Create input label and text box
    id_label = tk.Label(searcher_window, text="Enter ID to search for:", font=("Arial", 12), bg="#C88EA7")
    id_label.pack(pady=10)
    id_entry = tk.Entry(searcher_window, font=("Arial", 12), width=40)
    id_entry.pack(pady=10)

    # Create search button to trigger the search function
    search_button = tk.Button(searcher_window, text="Search", font=("Arial", 12), command=search_data)
    search_button.pack(pady=10)

    # Create result label and text box
    result_label = tk.Label(searcher_window, text="Search Results:", font=("Arial", 12), bg="#C88EA7")
    result_label.pack(pady=10)
    result_text = tk.Text(searcher_window, font=("Arial", 12), width=80, height=20, state='disabled')
    result_text.pack(pady=10)

    # Create close button
    close_button = tk.Button(searcher_window, text="Close", command=searcher_window.destroy)
    close_button.pack(pady=10)


##### find minumum id moldb weight

In [32]:
def execute_pipeline():
    pipeline = [
        {
            "$match": {
                "moldb_mono_mass": {
                    "$ne": float("nan"),
                    "$type": "double"
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "min_value": {
                    "$min": "$moldb_mono_mass"
                }
            }
        }
    ]
    result = all_toxin_data_collection.aggregate(pipeline)
    min_value = result.next()["min_value"]
    d = all_toxin_data_collection.find_one({'moldb_mono_mass': min_value})['_id']
    data = [
        {'_id': d, 'minimum value': min_value}
    ]
    df = pd.DataFrame(data)
    messagebox.showinfo("Result", df.to_string(index=False))


##### find minimum weight  

In [33]:
min_value = None
max_value = None

def find_min_weight():
    pipeline = [
        {
            "$match": {
                "weight": { "$regex": "^[0-9\.]+$" }
            }
        },
        {
            "$addFields": {
                "floatField": {
                    "$convert": {
                        "input": "$weight",
                        "to": "double"
                    }
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "minValue": { "$min": "$floatField" }
            }
        }
    ]

    result = all_toxin_data_collection.aggregate(pipeline)
    global min_value
    min_value = result.next()["minValue"]

def find_max_weight():
    pipeline = [
        {
            "$match": {
                "weight": { "$regex": "^[0-9\.]+$" }
            }
        },
        {
            "$addFields": {
                "floatField": {
                    "$convert": {
                        "input": "$weight",
                        "to": "double"
                    }
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "maxValue": { "$max": "$floatField" }
            }
        }
    ]

    result = all_toxin_data_collection.aggregate(pipeline)
    global max_value
    max_value = result.next()["maxValue"]

def show_message_box(value_type, value):
    messagebox.showinfo(f"Maximum {value_type}", f"The {value_type} value is: {value}")

def open_weight_window():
    # Create a new window for weight operations
    weight_window = tk.Toplevel(root)
    weight_window.title("Weight Operations")
    weight_window.configure(bg="#C88EA7")

    # Create button to find minimum weight
    min_button = tk.Button(weight_window, text="calculate Minimum Weight", command=find_min_weight)
    min_button.pack(pady=10)

    # Create button to find maximum weight
    max_button = tk.Button(weight_window, text="calculate Maximum Weight", command=find_max_weight)
    max_button.pack(pady=10)

    # Create button to show minimum weight
    min_value_button = tk.Button(weight_window, text="Show Minimum Weight", command=lambda: show_message_box("Minimum Weight", min_value))
    min_value_button.pack(pady=10)

    # Create button to show maximum weight
    max_value_button = tk.Button(weight_window, text="Show Maximum Weight", command=lambda: show_message_box("Maximum Weight", max_value))
    max_value_button.pack(pady=10)

    # Create close button for the weight operations window
    close_button = tk.Button(weight_window, text="Close", command=weight_window.destroy)
    close_button.pack(pady=10)

In [34]:
# Define function to handle button click event
def handle_button_click():
    x = all_toxin_data_collection.find({} , {"moldb_mono_mass":1}).sort("moldb_mono_mass",-1).limit(1)
    x = list(x)
    df = pd.DataFrame(x)
    result = str(df.at[0, "moldb_mono_mass"])
    messagebox.showinfo("Maximum Moldb Mono Mass", result)

def open_window():
    # Create a new window for the toxin data collection
    window = tk.Toplevel(root)
    window.title("Toxin Data Collection")
    window.geometry("400x300")
    window.configure(bg="light green")

    # Create button to find the maximum Moldb Mono Mass
    find_button = tk.Button(window, text="Find Maximum Moldb Mono Mass", command=handle_button_click)
    find_button.pack()
    
    find_id = tk.Button(window, text="Find minimumMoldb_id", command=open_execute_window)
    find_id.pack()
    
    
    close_btn=Button(window, text="Close", command=window.destroy)
    close_btn.pack()


##### find toxin type 

In [35]:
import tkinter as tk
import pymongo

def open_toxin_data_collection_gui():
    # Connect to MongoDB


    # Define function to handle button click event
    def handle_button_click():
        toxin_id = toxin_id_entry.get()
        documents = all_toxin_data_collection.find({"_id": toxin_id})
        toxin_types = ""
        for document in documents:
            toxin_types += document["toxin_structure"]["TYPES"] + "\n"
        toxin_types_text.delete(1.0, tk.END) # Clear previous result
        toxin_types_text.insert(tk.END, toxin_types) # Update text widget with new result

    # Create GUI window
    window = tk.Tk()
    window.title("Toxin Data Collection")
    window.geometry("400x300")
    window.configure(bg="#C88EA7")

    # Create GUI widgets
    toxin_id_label = tk.Label(window, text="Enter Toxin ID:")
    toxin_id_entry = tk.Entry(window)
    submit_button = tk.Button(window, text="Submit", command=handle_button_click)
    toxin_types_label = tk.Label(window, text="Toxin Types:")
    toxin_types_text = tk.Text(window, height=10, width=30)

    # Add widgets to GUI window
    toxin_id_label.pack()
    toxin_id_entry.pack()
    submit_button.pack()
    toxin_types_label.pack()
    toxin_types_text.pack()

    

##### find toxins smiles

In [36]:
import tkinter as tk
import pymongo

def open_toxin_data_smile_gui():
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["toxins"]
    all_toxin_data_collection = db["AllToxinData"]

    # Define function to handle button click event
    def handle_button_click():
        toxin_id = toxin_id_entry.get()
        documents = all_toxin_data_collection.find({"_id": toxin_id})
        toxin_smiles = ""
        for document in documents:
            toxin_smiles += document["toxin_structure"]["SMILES"] + "\n"
        toxin_smiles_text.delete(1.0, tk.END) # Clear previous result
        toxin_smiles_text.insert(tk.END, toxin_smiles) # Update text widget with new result

    # Create GUI window
    window = Toplevel(root)
    window.title("Toxin Data Collection")
    window.geometry("400x300")
    window.configure(bg="#C88EA7")

    # Create GUI widgets
    toxin_id_label = tk.Label(window, text="Enter Toxin ID:")
    toxin_id_entry = tk.Entry(window)
    submit_button = tk.Button(window, text="Submit", command=handle_button_click)
    toxin_smiles_label = tk.Label(window, text="Toxin SMILES:")
    toxin_smiles_text = tk.Text(window, height=10, width=30)

    # Add widgets to GUI window
    toxin_id_label.pack()
    toxin_id_entry.pack()
    submit_button.pack()
    toxin_smiles_label.pack()
    toxin_smiles_text.pack()




##### find toxins chem

In [37]:
import tkinter as tk
import pymongo

def open_toxin_data_chem_gui():

    # Define function to handle button click event
    def handle_button_click():
        toxin_id = toxin_id_entry.get()
        documents = all_toxin_data_collection.find({"_id": toxin_id})
        toxin_smiles = ""
        for document in documents:
            toxin_smiles += document["toxin_structure"]["SMILES"] + "\n"
        toxin_smiles_text.delete(1.0, tk.END) # Clear previous result
        toxin_smiles_text.insert(tk.END, toxin_smiles) # Update text widget with new result

    # Create GUI window
    window = Toplevel(root)
    window.title("Toxin Data Collection")
    window.geometry("400x300")
    window.configure(bg="#C88EA7")

    # Create GUI widgets
    toxin_id_label = tk.Label(window, text="Enter Toxin ID:")
    toxin_id_entry = tk.Entry(window)
    submit_button = tk.Button(window, text="Submit", command=handle_button_click)
    toxin_smiles_label = tk.Label(window, text="Toxin SMILES:")
    toxin_smiles_text = tk.Text(window, height=10, width=30)

    # Add widgets to GUI window
    toxin_id_label.pack()
    toxin_id_entry.pack()
    submit_button.pack()
    toxin_smiles_label.pack()
    toxin_smiles_text.pack()




##### find data  by molecular weight

In [38]:
import tkinter as tk
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB

# Define function to open the window
def open_toxin_molecular_weight_viewer():
    # Create tkinter GUI window
    window = Toplevel(root)
    window.title("Toxin Molecular Weight Viewer")
    window.configure(bg="#C88EA7")

    # Define the function to execute the query and display the result in a text box
    def execute_query():
        molecular_weight = float(input_text.get())
        documents = all_toxin_target_collection.find({"Molecular Weight": {"$gt": molecular_weight}}, {"_id": 0, 'NCBI Taxonomy Id': 1, 'UniProt Name': 1})
        result = list(documents)
        df = pd.DataFrame(result)
        result_text.configure(state='normal')
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, df.to_string(index=False))
        result_text.configure(state='disabled')

    # Create input label and text box
    input_label = tk.Label(window, text="Enter id:", font=("Arial", 12), bg="#C88EA7")
    input_label.pack(pady=10)
    input_text = tk.Entry(window, font=("Arial", 12), width=40)
    input_text.pack(pady=10)

    # Create button to execute query
    execute_button = tk.Button(window, text="Execute Query", command=execute_query)
    execute_button.pack(pady=10)

    # Create text box to display result
    result_text = tk.Text(window, font=("Arial", 12), bg="white", width=60, height=10, wrap=tk.WORD)
    result_text.pack(pady=10)


# Define function to open the Toxin Molecular Weight Viewer window
def open_molecular_weight_viewer_window():
    open_toxin_molecular_weight_viewer()



In [39]:
def open_find_queries_window():
    find_queries_window = tk.Toplevel(root)
    find_queries_window.title("Find Queries - allToxin_data")
    find_queries_window.geometry("600x400")

    # Set background image
    background_image = tk.PhotoImage(file="find.PNG")
    background_label = tk.Label(find_queries_window, image=background_image)
    background_label.place(x=0, y=0, relwidth=1, relheight=1)

    # Create buttons with the same height
    button_height = 2

    # Define the buttons in the desired order
    buttons = [
        ("Find State", find_State),
        ("Common Name Data", open_toxin_viewer),
        ("Find Toxin ID", open_toxin_searcher),
        ("Minimum Weight", open_weight_window),
        ("Maximum Moldb Mono Mass", open_window),
        ("Toxins SMILES", open_toxin_data_smile_gui),
        ("Toxins Chem", open_toxin_data_chem_gui),
        ("Find Toxin Type", open_toxin_data_collection_gui)
        ,("find_data_by_weight",open_molecular_weight_viewer_window)
    ]

    # Calculate the number of rows and columns for button placement
    num_rows = 3
    num_columns = (len(buttons) + num_rows - 1) // num_rows

    # Create and place buttons in the window
    for index, (button_text, button_command) in enumerate(buttons):
        row = index // num_columns
        column = index % num_columns

        button = tk.Button(find_queries_window, text=button_text, width=20, height=button_height, bg="#9c8d91", fg="white", command=button_command)
        button.place(relx=(column + 0.5) / num_columns, rely=(row + 0.3) / num_rows, anchor="center")

    # Create the close button
    close_button = tk.Button(find_queries_window, text="Close", width=15, height=button_height, bg="#9c8d91", fg="white", command=find_queries_window.destroy)
    close_button.place(relx=0.5, rely=0.9, anchor="center")



In [40]:
def center_window(window):
    window.update_idletasks()
    width = window.winfo_width()
    height = window.winfo_height()
    x = (window.winfo_screenwidth() // 2) - (width // 2)
    y = (window.winfo_screenheight() // 2) - (height // 2)
    window.geometry('{}x{}+{}+{}'.format(width, height, x, y))

def Queries():
    global Querywindow
    Querywindow = Toplevel(root)
    Querywindow.title("Query Window")
    Querywindow.geometry("800x500")
    center_window(Querywindow)
    background_image = PhotoImage(file="query.PNG")
    background_label = Label(Querywindow, image=background_image)
    background_label.image = background_image  # Retain a reference to the image
    background_label.pack()
    
    ##############Buttons######################
    
    button_width = 200
    button_height = 50


    
    find_mol = Button(Querywindow, text="Find Molecule", bg="gold", command=find_molecule)
    find_mol.pack()
    find_mol.place(relx=0.5, rely=0.55, anchor=CENTER, width=button_width, height=button_height)
    
    
    find_based_on_index = Button(Querywindow, text="Find Based on Index", bg="gold",command=indexing_find)
    find_based_on_index.pack()
    find_based_on_index.place(relx=0.5, rely=0.75, anchor=CENTER, width=button_width, height=button_height)

    close_button = Button(Querywindow, text="Close", width=10, command=close_query)
    close_button.pack()
    close_button.place(relx=0.5, rely=0.85, anchor=CENTER, width=button_width, height=button_height)



# all toxins target insertion

In [41]:
def update_toxinTarget():
    global updateWindow

    def update_data():
        # Retrieve the input values from the entry fields
        old_value = entry_old_value.get()
        new_value = entry_new_value.get()

        # Perform the update operation in the MongoDB collection
        try:
            # Update with your MongoDB connection details
            db = client['toxins']  # Replace 'your_database' with your database name
            collection = db["AllToxinTarget"]  # Replace 'your_collection' with your collection name
            collection.update_one({"Name": old_value}, {"$set": {"Name": new_value}})
            messagebox.showinfo("Success", "Data updated successfully!")
        except Exception as e:
            messagebox.showerror("Error",str(e))

    updateWindow = Toplevel(root)
    updateWindow.title("Update Data")
    updateWindow.geometry("400x300")
    updateWindow.configure(bg="lavender")  # Set background color to lavender

    # Create labels and entry fields for old and new values
    label_old_value = Label(updateWindow, text="Old Value:")
    label_old_value.pack()
    entry_old_value = Entry(updateWindow)
    entry_old_value.pack()

    label_new_value = Label(updateWindow, text="New Value:")
    label_new_value.pack()
    entry_new_value = Entry(updateWindow)
    entry_new_value.pack()

    # Create the update button
    button_update = Button(updateWindow, text="Update", command=update_data)
    button_update.pack()

    # Create the close button
    button_close = Button(updateWindow, text="Close", command=updateWindow.destroy)
    button_close.pack()



def open_targetWindow():
    insert_toxinTarget()

In [42]:
def update_toxin_data_collection(var1: str, var2: str) -> pd.DataFrame:
    # Connect to the MongoDB server and select the database and collection
    client = MongoClient("mongodb://localhost:27017/")
    db = client["toxins"]
    all_toxin_data_collection = db["AllToxinData"]

    # Update the documents with the old 'health_effects' value to the new value
    myquery = {"health_effects": var1 }
    newvalues = {"$set": {"health_effects": var2 } }
    all_toxin_data_collection.update_one(myquery, newvalues)

    # Find and return the documents with the new 'health_effects' value
    documents = all_toxin_data_collection.find({"health_effects": var2},{"health_effects":1}).limit(3)
    result = list(documents)
    df = pd.DataFrame(result)
    return df

def open_update_window_health_effect():
    # Create the top-level window for updating toxin data
    update_window = Toplevel(root)
    update_window.title("Update Toxin Data")
    update_window.configure(bg='light blue')
    update_window.geometry("600x400")

    # Create the labels, text boxes, and button for updating the toxin data
    old_health_effects_label =Label(update_window, text="Old Health Effects:", bg='light blue')
    old_health_effects_label.pack()

    old_health_effects_entry = Entry(update_window)
    old_health_effects_entry.pack()

    new_health_effects_label = Label(update_window, text="New Health Effects:", bg='light blue')
    new_health_effects_label.pack()

    new_health_effects_entry = Entry(update_window)
    new_health_effects_entry.pack()

    update_button = Button(update_window, text="Update", command=lambda: update_table())
    update_button.pack()

    def update_table():
        # Get the old and new health effects values and update the collection
        old_health_effects = old_health_effects_entry.get()
        new_health_effects = new_health_effects_entry.get()
        df = update_toxin_data_collection(old_health_effects, new_health_effects)

        # Clear the previous contents of the table frame
        for widget in table_frame.winfo_children():
            widget.destroy()

        # Create the table using pandas and display it in the table frame
        table = pd.DataFrame(df)
        table_widget = Text(table_frame, bg='white', relief="groove", bd=2)
        table_widget.insert("end", table.to_string(index=False))
        table_widget.pack()

    close_button = Button(update_window, text="Close", command=update_window.destroy)
    close_button.pack()

    # Create the label to show the updated documents
    results_label = Label(update_window, text="Results:", bg='light blue')
    results_label.pack()

    # Create the table frame to display the updated documents
    table_frame = Frame(update_window, bg='white', relief="groove", bd=2)
    table_frame.pack()

In [43]:
def update_synonms():
    # Create the GUI
    modify_synonms_window = Toplevel()
    modify_synonms_window.title("Update synonms")
    modify_synonms_window.configure(bg='light blue') # Set the background color to baby blue

    # Create the labels and text boxes for the 'old synonyms'and 'new synonyms' fields
    old_synonyms_label = Label(modify_synonms_window, text="Old Synonyms:", bg='light blue')
    old_synonyms_label.grid(row=0, column=0)
    old_synonyms_entry = Entry(modify_synonms_window, width=30)
    old_synonyms_entry.grid(row=0, column=1)

    new_synonyms_label = Label(modify_synonms_window, text="New Synonyms:", bg='light blue')
    new_synonyms_label.grid(row=1, column=0)
    new_synonyms_entry = Entry(modify_synonms_window, width=30)
    new_synonyms_entry.grid(row=1, column=1)

    # Create the update button
    update_button = Button(modify_synonms_window, text="Update", command=lambda: update_toxin_target_collection(old_synonyms_entry.get(), new_synonyms_entry.get()))
    update_button.grid(row=2, column=0, columnspan=2)
    close_button = Button(modify_synonms_window, text="Close", command=modify_synonms_window.destroy)
    close_button.grid(row=3, column=0, columnspan=2)

##### update date 

In [44]:
import tkinter as tk
from pymongo import MongoClient
import pandas as pd

def update_toxin_data(var1: str, var2: str) -> pd.DataFrame:
    # Connect to the MongoDB server and select the database and collection
    client = MongoClient("mongodb://localhost:27017/")
    db = client["toxins"]
    all_toxin_data_collection = db["AllToxinData"]

    # Find the document with the given _id and retrieve its current updated_at value
    document = all_toxin_data_collection.find_one({"_id": var1})

    if document is not None:
        old_updated_at = document.get("updated_at")

        # Update the document with the new updated_at value
        newvalues = {"$set": {"updated_at": var2}}
        all_toxin_data_collection.update_one({"_id": var1}, newvalues)

        # Retrieve the updated document
        updated_document = all_toxin_data_collection.find_one({"_id": var1})

        # Prepare the data to create a DataFrame
        data = [
            {"_id": updated_document.get("_id"), "old_updated_at": old_updated_at, "new_updated_at": updated_document.get("updated_at")}
        ]
        df = pd.DataFrame(data)
        return df
    else:
        # Return an empty DataFrame if the document is not found
        return pd.DataFrame()

def update_toxin_data_gui():
    update_date = tk.Toplevel(root)

    def update_data():
        var1 = id_entry.get()
        var2 = new_updated_at_entry.get()
        df = update_toxin_data(var1, var2)
        result_text.delete("1.0", tk.END)
        result_text.insert(tk.END, df.to_string(index=False))

    # Create the labels and entry fields
    id_label = tk.Label(update_date, text="Enter _id:", bg="light blue")
    id_label.grid(row=0, column=0, padx=5, pady=5)
    id_entry = tk.Entry(update_date)
    id_entry.grid(row=0, column=1, padx=5, pady=5)

    new_updated_at_label = tk.Label(update_date, text="Enter new updated_at:", bg="light blue")
    new_updated_at_label.grid(row=1, column=0, padx=5, pady=5)
    new_updated_at_entry = tk.Entry(update_date)
    new_updated_at_entry.grid(row=1, column=1, padx=5, pady=5)

    # Create the update button
    update_button = tk.Button(update_date, text="Update", command=update_data)
    update_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

    # Create the result label and text box
    result_label = tk.Label(update_date, text="Result:", bg="light blue")
    result_label.grid(row=3, column=0, padx=5, pady=5)
    result_text = tk.Text(update_date, height=5, width=60)
    result_text.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

##### update locus value

In [45]:
def update_toxin_target_collection(var1: str, var2: str) -> pd.DataFrame:
    # Connect to the MongoDB server and select the database and collection
    client = MongoClient("mongodb://localhost:27017/")
    db = client["toxins"]
    all_toxin_target_collection = db["AllToxinTarget"]

    # Update the documents with the old 'Locus' value to the new value
    myquery = {"Locus": var1 }
    newvalues = {"$set": {"Locus": var2 } }
    all_toxin_target_collection.update_one(myquery, newvalues)

    # Find and return the documents with the new 'Locus' value
    documents = all_toxin_target_collection.find({"Locus":var2})
    result = list(documents)
    df = pd.DataFrame(result)
    return df

def update_toxin_target_gui():
    # Create the GUI
    update_locus = Toplevel()
    update_locus.title("Update Toxin Target")
    update_locus.configure(bg='light blue') # Set the background color to baby blue    # Create the labels and text boxes for the 'old Locus' and 'new Locus' values
    old_locus_label = Label(update_locus, text="Old Locus:", bg='light blue')
    old_locus_label.grid(row=0, column=0, padx=5, pady=5)
    old_locus_entry = Entry(update_locus)
    old_locus_entry.grid(row=0, column=1, padx=5, pady=5)

    new_locus_label = Label(update_locus, text="New Locus:", bg='light blue')
    new_locus_label.grid(row=1, column=0, padx=5, pady=5)
    new_locus_entry = Entry(update_locus)
    new_locus_entry.grid(row=1, column=1, padx=5, pady=5)

    # Create the button to update the collection
    update_button = Button(update_locus, text="Update", bg='white', command=lambda: update_table())
    update_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

    # Create the label to show the updated documents
    results_label = Label(update_locus, text="Results:", bg='light blue')
    results_label.grid(row=3, column=0, padx=5, pady=5)

    # Create the table to display the updated documents
    table_frame = Frame(update_locus, bg='white', relief="groove", bd=2)
    table_frame.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

    def update_table():
        
        # Get the old and new Locus values and update the collection
        old_locus = old_locus_entry.get()
        new_locus = new_locus_entry.get()
        df = update_toxin_target_collection(old_locus, new_locus)

        # Clear the previous contents of the table frame
        for widget in table_frame.winfo_children():
            widget.destroy()

        if not df.empty:
            # Select only the "Locus" and "_id" columns
            df_filtered = df[["Locus", "_id"]]

            # Create the table using pandas and display it in the table frame
            table_widget = Text(table_frame, bg='white', relief="groove", bd=2)
            table_widget.insert("end", df_filtered.to_string(index=False))
            table_widget.pack()
        else:
            no_results_label = Label(table_frame, text="No results found.", bg='white', relief="groove", bd=2)
            no_results_label.pack()

In [46]:
def open_update_target_data():
    # Create the top-level window
    update_window_target = Toplevel(root)
    update_window_target.title("Update Target Data")
    update_window_target.configure(bg='steelblue4')

    # Set the width and height of the window
    window_width = 400
    window_height = 300
    update_window_target.geometry(f"{window_width}x{window_height}")

    # Create the buttons
    button1 = Button(update_window_target, text="Update Synonyms", command=open_update_screen)
    button1.pack(pady=10)

    button2 = Button(update_window_target, text="Update Locus", command=update_toxin_target_gui)
    button2.pack(pady=10)

    # Create the close button
    close_button = Button(update_window_target, text="Close", command=update_window_target.destroy)
    close_button.pack(pady=10)


In [47]:
def open_update_window_entry():
    # Create the top-level window
    update_window = Toplevel(root)
    update_window.title("Update Window")
    update_window.configure(bg='steelblue4')

    # Set the width and height of the window
    window_width = 400
    window_height = 300
    update_window.geometry(f"{window_width}x{window_height}")

    # Create the buttons
    button1 = Button(update_window, text="Open Update Target Data", command=open_update_target_data)
    button1.pack(pady=10)

    button2 = Button(update_window, text="Open Update Toxin Data", command=open_update_window)
    button2.pack(pady=10)

    close_button = Button(update_window, text="Close", command=update_window.destroy)
    close_button.pack(pady=10)


In [48]:
def insert_toxinTarget():
    global targetWindow

    def insert_data():
        # Retrieve the input values from the entry fields
        _id = entry_id.get()
        name = entry_name.get()
        uniprot_name = entry_uniprot.get()
        organism = entry_organism.get()
        taxonomy_id = entry_taxonomy.get()
        gene_name = entry_gene.get()

        # Create a dictionary with the input values
        data = {
            "_id": _id,
            "Name": name,
            "UniProt Name": uniprot_name,
            "Organism": organism,
            "NCBI Taxonomy Id": taxonomy_id,
            "Gene Name": gene_name
        }

        # Insert the data into the MongoDB collection
        try:
            # Update with your MongoDB connection details
            db = client['toxins']  # Replace 'your_database' with your database name
            collection = db["AllToxinTarget"]  # Replace 'your_collection' with your collection name
            collection.insert_one(data)
            messagebox.showinfo("Success", "Data inserted successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def open_update_window():
        
        update_toxinTarget()

    targetWindow = Toplevel(root)
    targetWindow.title("Insert Data")
    targetWindow.geometry("500x400")
    targetWindow.configure(bg="lavender")  # Set background color to lavender

    # Create labels and entry fields for each value
    label_id = Label(targetWindow, text="ID:")
    label_id.pack(anchor="center")
    entry_id = Entry(targetWindow)
    entry_id.pack(anchor="center")

    label_name = Label(targetWindow, text="Name:")
    label_name.pack(anchor="center")
    entry_name = Entry(targetWindow)
    entry_name.pack(anchor="center")

    label_uniprot = Label(targetWindow, text="UniProt Name:")
    label_uniprot.pack(anchor="center")
    entry_uniprot = Entry(targetWindow)
    entry_uniprot.pack(anchor="center")

    label_organism = Label(targetWindow, text="Organism:")
    label_organism.pack(anchor="center")
    entry_organism = Entry(targetWindow)
    entry_organism.pack(anchor="center")

    label_taxonomy = Label(targetWindow, text="NCBI Taxonomy Id:")
    label_taxonomy.pack(anchor="center")
    entry_taxonomy = Entry(targetWindow)
    entry_taxonomy.pack(anchor="center")

    label_gene = Label(targetWindow, text="Gene Name:")
    label_gene.pack(anchor="center")
    entry_gene = Entry(targetWindow)
    entry_gene.pack(anchor="center")

    # Create the insert button
    button_insert = Button(targetWindow, text="Insert", command=insert_data)
    button_insert.pack(anchor="center")

    # Create the update button
    button_update = Button(targetWindow, text="Update", command=open_update_target_data)
    button_update.pack(anchor="center")
    
    find_btn=Button(targetWindow,text="find Queries",command=open_find_queries_window_target)
    find_btn.pack(anchor="center")

    # Create the close button
    button_close = Button(targetWindow, text="Close", command=targetWindow.destroy)
    button_close.pack(anchor="center")


# Data Analysis Windows

In [49]:

def close_window():

    graph_window.destroy()
    
def close_cloud():
    cloud_window.destroy()
    
def close_query():
    Querywindow.destroy()  

In [50]:
def open_dataAnalysis():
    # Create a new Toplevel window for the new graph
    data_AnalysisWindow = Toplevel(root)
    data_AnalysisWindow.title("Data Analysis Window")
    data_AnalysisWindow.geometry("800x600")
    #data_AnalysisWindow.configure(bg="blue4")
    background_image = PhotoImage(file="graphs.PNG")
    background_label = Label(data_AnalysisWindow, image=background_image)
    background_label.image = background_image  # Retain a reference to the image
    background_label.pack()
    
    #################Show scatter plot########################
    def show_graph():
        global graph_window
        # Create a new Toplevel window
        graph_window =Toplevel(root)
        graph_window.configure(bg="gold")
        graph_window.title("Graph")
        # Create a Figure object
        fig = Figure(figsize=(6, 4), dpi=100)
        # Add a subplot to the figure
        ax = fig.add_subplot(111)
        df = pd.read_csv('AllToxinTarget.csv')
        col1 = 'Genbank Protein ID'  # the first column containing the data for the x-axis
        col2 = 'Number of Residues'  # the second column containing the data for the y-axis
        # select the first 10 rows of the columns
        data1 = df.head(20)[col1]
        data2 = df.head(20)[col2]

        # Plot your data (example: a simple line plot)

        ax.scatter(data1, data2)
        ax.set_xlabel('Genbank Protein ID')
        ax.set_ylabel('Number of Residues')
        ax.set_title("Genbank ID and Number of Residues")

        # Create a canvas for displaying the figure
        canvas = FigureCanvasTkAgg(fig, master=graph_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

        close_button =Button(graph_window,text="Close",width=10, command=close_window)
        close_button.pack()
        
    ########################draw theortical phi graph #########################
    def draw_theoretical_phi():
        
        # Create a new tkinter window
        global draw_theoretical_phi_window

        draw_theoretical_phi_window = Toplevel(root)
        draw_theoretical_phi_window.title("Theoretical Pi by NCBI Taxonomy Id")

        # Load the CSV file into a pandas dataframe
        df = pd.read_csv('AllToxinTarget.csv')
        df = df.head(40)

        # Convert NCBI Taxonomy Id column to string and Theoretical Pi column to float
        df['NCBI Taxonomy Id'] = df['NCBI Taxonomy Id'].astype(str)
        df['Theoretical Pi'] = pd.to_numeric(df['Theoretical Pi'], errors='coerce')

        # Create a Figure object
        fig = plt.Figure(figsize=(6, 4), dpi=100)

        # Create a subplot in the figure
        ax = fig.add_subplot(111)

        # Create a hexbin plot using matplotlib
        ax.hexbin(df['NCBI Taxonomy Id'], df['Theoretical Pi'], gridsize=20)

        # Add title and labels
        ax.set_title('Theoretical Pi by NCBI Taxonomy Id (First 10 values)')
        ax.set_xlabel('NCBI Taxonomy Id')
        ax.set_ylabel('Theoretical Pi')

        # Create a canvas for displaying the plot
        canvas = FigureCanvasTkAgg(fig, master=draw_theoretical_phi_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

        # Create a function to close the window
        def close_window():
            draw_theoretical_phi_window.destroy()

        # Create a close button
        close_button = Button(draw_theoretical_phi_window, text="Close", command=close_window)
        close_button.pack()
        
    ########################## barplott function####################################
    def show_barplot():
        
        global barwindow
        # Create a new tkinter window
        barwindow = Toplevel(root)
        root.title("Toxin State Countplot")

        # Load the CSV file into a pandas dataframe
        df = pd.read_csv('AllToxinData.csv')
        df = df.head(10)

        # Create a Figure object
        fig = plt.Figure(figsize=(6, 4), dpi=100)

        # Create a subplot in the figure
        ax = fig.add_subplot(111)

        # Create a countplot using seaborn with custom color palette
        sns.countplot(x='common_name', hue='state', data=df, palette=['purple', 'yellow', 'blue'], ax=ax)

        # Rotate the x-axis labels to avoid overlap
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

        # Add title and labels
        ax.set_title('Toxin State by Name')
        ax.set_xlabel('Toxin Name')
        ax.set_ylabel('Count')

        # Create a canvas for displaying the plot
        canvas = FigureCanvasTkAgg(fig, master=barwindow)
        canvas.draw()
        canvas.get_tk_widget().pack()

        # Create a function to close the window
        def close_window():
            barwindow.destroy()

        # Create a close button
        close_button =Button(barwindow, text="Close", command=close_window)
        close_button.pack()

############################################show box plot#####################################
    def show_box_plots():
        global boxplot_window
        boxplot_window=Toplevel(root)
        # Load the data from the CSV file
        df = pd.read_csv('AllToxinData.csv')

        # Specify the columns to use for the box plots
        col1 = 'weight'
        col2 = 'moldb_average_mass'
        col3 = 'moldb_mono_mass'

        # Select the first 10 rows of the columns and parse the data as floats
        data1 = df.head(10)[col1].astype(float)
        data2 = df.head(10)[col2].astype(float)
        data3 = df.head(10)[col3].astype(float)

        # Create the subplots for the box plots
        fig, axs = plt.subplots(1, 3, figsize=(10, 5))

        # Create a box plot for each column using matplotlib
        axs[0].boxplot(data1)
        axs[1].boxplot(data2)
        axs[2].boxplot(data3)

        # Add titles and labels for each box plot
        axs[0].set_title('moldb_average_mass')
        axs[1].set_title('moldb_mono_mass')
        axs[2].set_title('weight')
        axs[0].set_xlabel('X-axis Label')
        axs[1].set_xlabel('X-axis Label')
        axs[2].set_xlabel('X-axis Label')
        axs[0].set_ylabel('Y-axis Label')
        axs[1].set_ylabel('Y-axis Label')
        axs[2].set_ylabel('Y-axis Label')

        # Adjust the layout and spacing of the subplots
        fig.tight_layout(pad=3.0)

        canvas = FigureCanvasTkAgg(fig, master=boxplot_window)
        canvas.draw()
        canvas.get_tk_widget().pack()
        def close_boxwindow():
            boxplot_window.destroy()

        # Create a Button to close the window
        close_button = Button(boxplot_window, text="Close", command=close_boxwindow)
        close_button.pack()
        
    ##############################draw stack plot##################################################33
    def draw_area_plot():
        # Create a new tkinter window
        global areaplot_window
        areaplot_window = Toplevel(root)
        areaplot_window.title("Area Plot")

        # Load the CSV file into a pandas dataframe
        df = pd.read_csv('AllToxinData.csv')
        df = df.head(10)

        # Create a Figure object
        fig = plt.Figure(figsize=(6, 4), dpi=100)

        # Create a subplot in the figure
        ax = fig.add_subplot(111)

        # Convert toxicity and name columns to string
        df['chemical_formula'] = df['chemical_formula'].astype(str)
        df['common_name'] = df['common_name'].astype(str)

        # Create an area plot using matplotlib
        ax.stackplot(df['common_name'], df['chemical_formula'], alpha=0.7)

        # Add title and labels
        ax.set_title('Chemical Formula')
        ax.set_xlabel('Name')
        ax.set_ylabel('Toxicity')

        # Create a canvas for displaying the plot
        canvas = FigureCanvasTkAgg(fig, master=areaplot_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

        # Create a function to close the window
        def close_window():
            areaplot_window.destroy()

        # Create a close button
        close_button =Button(areaplot_window, text="Close", command=close_window)
        close_button.pack()
        #########################################word cloud button######################################3
    def show_wordcloud():
        global cloud_window
        # Generate the word cloud
        fields = ['Toxin Name']
        mechanism= pd.read_csv('Relations_table1.csv', usecols=fields)
        text2=mechanism["Toxin Name"]
        mechanism=text2[2:1500]
        text3 = ' '.join(text2)

        #wc=WordCloud(max_words=50,background_color="white").generate(text3)  # Replace with your text data
        wordcloud = WordCloud(background_color="white").generate(text3)

        # Create a Toplevel window
        cloud_window = Toplevel(root)
        cloud_window.title("Word Cloud")

        # Convert the word cloud image to a Tkinter-compatible format
        image = wordcloud.to_image()

        # Display the word cloud image in a Tkinter Label widget
        label =Label(cloud_window)
        label.pack()

        # Convert the image to PhotoImage format and set it as the Label's image
        photo = ImageTk.PhotoImage(image)
        label.configure(image=photo)
        label.image = photo
        close_button =Button(cloud_window, text="Close",width=10, command=close_cloud)
        close_button.pack()


    # Create a button to show the word cloud
    word_cloud_btn = Button(data_AnalysisWindow, text="Show Word Cloud",bg="gold" ,command=show_wordcloud)
    word_cloud_btn.pack()
    
    #create a button  to show box plot
    boxplot_btn = Button(data_AnalysisWindow, text="Show box plot",bg="gold" ,command=show_box_plots)
    boxplot_btn.pack()
    
    #create a button to show draw_theoretical_phi graph
    
    draw_theoretical_phi_btn=Button(data_AnalysisWindow, text="Show theoretical_phi graph",bg="gold" ,command=draw_theoretical_phi)
    draw_theoretical_phi_btn.pack()




    # Create a button to show the scatter plot
    scatter_button = Button(data_AnalysisWindow, text="Show Scatter Plot",bg="gold", command=show_graph)
    scatter_button.pack()
    #create button to show barplot##################
    show_barplot_btn=Button(data_AnalysisWindow, text="Show bar Plot",bg="gold", command=show_barplot)
    show_barplot_btn.pack()
    
    #create a button to show a stack plot
    draw_area_plot_btn=Button(data_AnalysisWindow, text="Show area plot",bg="gold", command=draw_area_plot)
    draw_area_plot_btn.pack()
    


    # Calculate the position for the button
    button_width = 200
    button_height = 50
    button_x = (800 - button_width) // 2
    button_y = (600 - button_height) // 2

    # Create the button
    scatter_button.place(x=button_x, y=button_y, width=button_width, height=button_height)
    word_cloud_btn.place(x=button_x, y=button_y+50, width=button_width, height=button_height)
    boxplot_btn.place(x=button_x, y=button_y+100, width=button_width, height=button_height)
    show_barplot_btn.place(x=button_x, y=button_y+150, width=button_width, height=button_height)
    draw_theoretical_phi_btn.place(x=button_x, y=button_y+200, width=button_width, height=button_height)
    draw_area_plot_btn.place(x=button_x, y=button_y-50, width=button_width, height=button_height)
        

    # Handle the window close event
    def on_close():
        if messagebox.askokcancel("Close", "Do you want to close the Data Analysis window?"):
            data_AnalysisWindow.destroy()

    data_AnalysisWindow.protocol("WM_DELETE_WINDOW", on_close)

In [51]:
import tkinter as tk
from pymongo import MongoClient
from pymongo.cursor import Cursor

# connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['toxins']
collection = db['mycollection']

def open_code_executor_window():
    code_executor_window = tk.Toplevel(root)
    code_executor_window.title("Code Executor")

    code_label = tk.Label(code_executor_window, text='Enter code:')
    code_label.pack()

    code_entry = tk.Entry(code_executor_window, width=50)
    code_entry.pack()

    execute_button = tk.Button(code_executor_window, text='Execute', command=lambda: execute_code(code_entry, result_text))
    execute_button.pack()

    result_label = tk.Label(code_executor_window, text='Result:')
    result_label.pack()

    result_frame = tk.Frame(code_executor_window, bg='gray')
    result_frame.pack(fill=tk.BOTH, expand=True)

    result_scrollbar = tk.Scrollbar(result_frame)
    result_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    result_text = tk.Text(result_frame, bg='lavender', fg='black', state=tk.DISABLED,
                          yscrollcommand=result_scrollbar.set)
    result_text.pack(fill=tk.BOTH, expand=True)

    result_scrollbar.config(command=result_text.yview)

def execute_code(code_entry, result_text):
    code = code_entry.get()
    try:
        # check if code contains a query to the database
        if 'db.' in code:
            result = eval(code, {'db': db, 'collection': collection})
        else:
            result = eval(code)

        result_text.config(state=tk.NORMAL)
        result_text.delete('1.0', tk.END)

        # check if result is a cursor object from MongoDB
        if isinstance(result, Cursor):
            for document in result:
                result_text.insert(tk.END, str(document) + '\n')
        else:
            result_text.insert(tk.END, str(result))

        result_text.config(state=tk.DISABLED)
    except Exception as e:
        result_text.config(state=tk.NORMAL)
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, str(e))
        result_text.config(state=tk.DISABLED)




# Main Screen

In [ ]:
from tkinter import Tk, Label, Button
from PIL import Image, ImageTk

root = Tk()

# Create a photoimage object of the image in the path
image1 = Image.open("homeScreen.PNG")
test = ImageTk.PhotoImage(image1)

root.title("Toxins database")

label1 = Label(image=test)
label1.image = test
label1.place(x=0, y=0)

# Define functions for button commands (e.g., open_dataAnalysis, insertion_screen)

Button(root, width=25, height=2, text="Data Analysis", background="SpringGreen3", foreground="blue4",
       font=('Helvetica 13 bold'), command=open_dataAnalysis).place(x=590, y=500)
Button(root, width=25, height=2, text="Execute your  Query ", background="SpringGreen3", foreground="blue4",
       font=('Helvetica 13 bold'), command=open_code_executor_window).place(x=300, y=500)

Button(root, width=25, height=2, text="Data query", background="SpringGreen3", foreground="blue4",
       font=('Helvetica 13 bold'), command=query_window).place(x=10, y=500)

root.geometry("850x600")
root.mainloop()


##### update date all toxin data

In [ ]:
"Molecular Weight"